# Incremental Data Loading using AutoLoader
- **Autoloader** --> 
✅ directory listing 
✅ file notification 


In [0]:
%sql
CREATE SCHEMA Carbon_catalog.net_schema;

In [0]:
checkpoint_location = "abfss://silver@storagecarbonproject.dfs.core.windows.net/checkpoint" #checkpoint path

In [0]:
df = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "csv")\
  .option("cloudFiles.schemaLocation",checkpoint_location)\
  .load("abfss://raw@storagecarbonproject.dfs.core.windows.net/") #cargando datos desde raw (archivo .csv de origen)

In [0]:
display(df)

Name,Carbon Dioxide Emissions,_rescued_data
ShortName,CO2 Emissions,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
null,null,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
null,null,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
null,null,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
Theme,Environment,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
Sub-theme,Air,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
null,null,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
Title,Carbon Dioxide Emissions,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
Description,"Local and Regional CO2 Emissions Estimates for 2005-2011 plus, subset data of CO2 emissions within the scope of influence of Local Authorities (previously called National Indicator 186: Per capita CO2 emissions in the LA area) by sector","{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"
Subject,Emissions,"{""_c11"":"""",""_c3"":"""",""_c5"":"""",""_c10"":"""",""_c4"":"""",""_c9"":"""",""_c8"":"""",""_file_path"":""abfss://raw@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough.csv"",""_c7"":"""",""_c2"":"""",""_c6"":""""}"


In [0]:
from pyspark.sql.functions import col

# Renombro las columnas problematicas, cuando los indices viene con espacios en blanco, ya que no son aceptados. ✅
df = df.select([col(c).alias(c.replace(" ", "_")) for c in df.columns])

df.writeStream \
    .option("checkpointLocation", checkpoint_location) \
    .trigger(processingTime="10 seconds") \
    .start("abfss://bronze@storagecarbonproject.dfs.core.windows.net/carbon-emissions-borough") # escrito en delta format ✅